# Multi-step tool calling: Manual first, then Automatic (Agent)

This notebook demonstrates manual multi-tool orchestration first, followed by automatic (agent) orchestration, using two tools. This is designed for students to experiment with multi-step function calling in LangChain using OCI Generative AI.

## What this notebook does
Demonstrates manual multi-step function calling with custom tools using OCI Generative AI for LLM, followed by automatic agent orchestration.

## Documentation to reference
- OCI Gen AI: https://docs.oracle.com/en-us/iaas/Content/generative-ai/pretrained-models.htm
- LangChain: https://docs.langchain.com/oss/python/langchain/agents
- How to build tools: https://python.langchain.com/docs/how_to/custom_tools/
- OCI OpenAI compatible SDK: https://github.com/oracle-samples/oci-openai  note: supports OpenAI, XAI & Meta models. Also supports OpenAI Responses API 
- OCI langchain SDK: https://github.com/oracle-devrel/langchain-oci-genai  note: as of Nov 2025 it is not compatible with langchain v1.0. supports all OCI models including Cohere
- OCI GenAI SDK: https://github.com/oracle/oci-python-sdk/tree/master/src/oci/generative_ai_inference/models

## Relevant slack channels
- #generative-ai-users: *for questions on OCI Gen AI* 
- #igiu-innovation-lab: *general discussions on your project* 
- #igiu-ai-learning: *help with sandbox environment or help with running this code* 

## Env setup
- sandbox.yaml: Contains OCI config, compartment, DB details, and wallet path.
- .env: Load environment variables (e.g., API keys if needed).
- configure cwd for jupyter match your workspace python code: 
    -  vscode menu -> Settings > Extensions > Jupyter > Notebook File Root
    -  change from `${fileDirname}` to `${workspaceFolder}`

## How to run the notebook
Run the cells in order. Make sure your sandbox.yaml is configured. You can also run equivalent Python files like langChain/function_calling/langchain_multi_step.py or langChain/function_calling/langchain_multi_manual.py using `uv run <path>`.

## Comments to important sections of notebook
- Cell 1: Instantiate the client and load config
- Cell 2: Define the tools
- Manual section: Demonstrates manual multi-tool orchestration in a loop
- Automatic section: Shows agent-based multi-step tool calling

## Experimentation Tips
- Try changing the LLM_MODEL to different values like 'meta.llama-3.1-405b-instruct' or 'xai.grok-4' to see how responses vary.
- Experiment with the system_prompt in the agent: modify it to change the personality or add more instructions.
- Add more tools or modify existing ones to see how the agent handles them.
- Try different reasoning parameters in the client (e.g., change 'effort' to 'high').

## 1. Instantiate the client

This cell sets up the OCI OpenAI client using your configuration. It loads the sandbox.yaml file, sets the model (try experimenting with different models!), and initializes the LLM client with reasoning parameters for multi-step tasks.

In [ ]:
import os, sys
from dotenv import load_dotenv
from envyaml import EnvYAML
from langchain.tools import tool
from langchain_core.messages import HumanMessage
from langchain.agents import create_agent

from langChain.oci_openai_helper import OCIOpenAIHelper
load_dotenv()

def load_config(config_path):
    try:
        with open(config_path, 'r') as f:
            return EnvYAML(config_path)
    except FileNotFoundError:
        print(f"Error: Configuration file '{config_path}' not found.")
        return None

def pretty_print(response):
    for i, m in enumerate(response["messages"], 1):
        role = getattr(m, "type", m.__class__.__name__)
        content = m.content if isinstance(m.content, str) else str(m.content)
        print(f"{i:>2}. [{role.upper()}] {content}")

SANDBOX_CONFIG_FILE = "sandbox.yaml"
LLM_MODEL = "openai.gpt-5"

scfg = load_config(SANDBOX_CONFIG_FILE)
llm = OCIOpenAIHelper.get_langchain_openai_client(
    model_name=LLM_MODEL,
    config=scfg,
    use_responses_api=True,
    reasoning={"effort": "low", "summary": "auto"}
)


## 2. Define the tools

Here we define two custom tools: one for getting weather and one for projecting bills. These are LangChain tools decorated with @tool. Experiment by adding more tools or modifying their logic!

In [ ]:
from langchain.tools import tool

@tool
def get_weather(city: str) -> str:
    """Gets the weather for a given city"""
    return f"The weather in {city} is 70 Fahrenheit"

@tool
def get_projection_bill(current_bill: int, gas_oven: bool) -> int:
    """Returns the projected bill depending on current bill and oven usage"""
    if gas_oven:
        return current_bill + 45
    return current_bill + 4

tools = [get_weather, get_projection_bill]
tool_map = {t.name.lower(): t for t in tools}


## Manual multi-tool orchestration

This section shows how to manually handle multi-step tool calling. The model may decide to call tools, and we loop until it finishes. This gives you control over the process. Notice how the model might ask for clarification (like oven type) before using tools.

In [ ]:
messages = [
    HumanMessage(
        "Which will be my projected bill? I'm in San Francisco, and I have oven. My past bill was $45"
    )
]

llm_with_tools = llm.bind_tools([get_weather, get_projection_bill])

while True:
    ai_message = llm_with_tools.invoke(messages)
    print("\nAI response content:", ai_message.content)
    tool_calls = getattr(ai_message, "tool_calls", None)
    print("Tool calls:", tool_calls)

    if not tool_calls:
        print("\nNo tool calls detected — conversation finished.")
        final_message = ai_message
        break

    messages.append(ai_message)

    for tool_call in tool_calls:
        tool_name = tool_call["name"].lower()
        selected_tool = tool_map.get(tool_name)
        if not selected_tool:
            print(f"Unknown tool requested: {tool_name}")
            continue
        print(f"Executing tool: {tool_name} with args: {tool_call.get('args', {})}")
        tool_msg = selected_tool.invoke(tool_call)
        print("Tool result:", tool_msg.content)
        messages.append(tool_msg)

print("\n************************ Conversation Ended ************************")
print("Final model message:\n", final_message.content)


## Automatic (Agent) multi-step orchestration

Now we use LangChain's agent to handle tool calling automatically. The agent decides when to call tools and how to respond. Notice how it can handle multiple steps and even parallel tool calls (like calling get_projection_bill twice for gas and electric scenarios).

In [ ]:
agent = create_agent(
    llm,
    tools=[get_weather, get_projection_bill],
    system_prompt="use one or more tools to get an answer; reply in Hinglish (Hindi mixed with English)"
)

messages_agent = [
    HumanMessage(
        content="Which will be my projected bill? I'm in San Francisco, and I have oven. My past bill was $45"
    )
]

print("************************** Agent Multi-Step invoke and details **************************")
response = agent.invoke({"messages": messages_agent})
pretty_print(response)

print("\n************************** Agent Multi-Step stream **************************")
for chunk in agent.invoke({"messages": messages_agent}, stream_mode="updates"):
    for step, data in chunk.items():
        print(f"step: {step}", flush=True)
        print(f"content: {data['messages'][-1].content_blocks}", flush=True)


## Exercises

1. **Modify the tools**: Add a third tool, like `get_population(city: str) -> int`, and update the agent to use it. Test with a query like "What's the weather and population in San Francisco?"

2. **Change the system prompt**: Experiment with different personalities, e.g., "reply as a strict teacher" or "reply in Shakespearean English". How does it affect tool usage?

3. **Parallel tool calls**: In the manual section, modify the code to handle parallel tool calls if the model suggests them.

4. **Error handling**: Add error handling in the tool functions, e.g., if `current_bill` is negative, raise an exception. See how the agent responds.

5. **Streaming vs. non-streaming**: Compare the streaming output with the regular invoke. What differences do you notice in multi-step scenarios?

6. **Model comparison**: Run the notebook with different LLM_MODEL values and compare the reasoning and tool usage.